## Notebook to Check the Normalized and YOLO Prepared Bounding Boxes Visually

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import cv2

In [ ]:
base_dir = "/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01"  
image_dirs = [os.path.join(base_dir, "images", subdir) for subdir in ["train", "val", "test"]]
label_dirs = [os.path.join(base_dir, "labels", subdir) for subdir in ["train", "val", "test"]]

image_paths = []
for dir_path in image_dirs:
    for fname in os.listdir(dir_path):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            image_paths.append(os.path.join(dir_path, fname))

random_images = random.sample(image_paths, 10)

def draw_yolo_boxes(image_path):
    for image_root, label_root in zip(image_dirs, label_dirs):
        if image_path.startswith(image_root):
            label_path = image_path.replace(image_root, label_root).rsplit('.', 1)[0] + '.txt'
            break
    else:
        print(f"No label found for {image_path}")
        return None

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img.shape

    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls, x_center, y_center, bw, bh = map(float, parts)
                x1 = int((x_center - bw / 2) * w)
                y1 = int((y_center - bh / 2) * h)
                x2 = int((x_center + bw / 2) * w)
                y2 = int((y_center + bh / 2) * h)
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return img

plt.figure(figsize=(10, 100))
for i, image_path in enumerate(random_images):
    img = draw_yolo_boxes(image_path)
    if img is not None:
        plt.subplot(10, 1, i + 1)
        plt.imshow(img)
        plt.title(os.path.basename(image_path))
        plt.axis('off')

plt.tight_layout()
plt.show()